In [ ]:
import pandas as pd
import numpy as np
import random
from copy import deepcopy as copy

In [ ]:
dice_values = [(4, 1), (6, 2), (8, 3), (10, 4), (10, 4), (12, 5), (20, 6)]
goal = 20
player_1_total = 0
player_2_total = 0

In [ ]:
player_1_money = 100
player_2_money = 100

In [ ]:
data = pd.DataFrame(columns=["round", "player 1 strategy", 
                             "player 2 strategy", 
                             "player 1 result", 
                             "player 2 result", 
                             "player 1 money", 
                             "player 2 money"])

In [ ]:
class Player:
    def __init__(self, name, strategy, money, active):
        self.name = name
        self.strategy = strategy
        self.money = money
        self.active = active
        self.total = 0
    def __repr__(self):
        return self.name
class Die:
    def __init__(self, value, cost):
        self.value = value
        self.cost = cost
    def __repr__(self):
        return str(self.value)
original_dice_list = [Die(x, y) for x, y in dice_values]
original_dice_list

In [ ]:
def StartHighNeverBustStrategy(total, goal, money, dice):
    dice.sort(reverse=True, key=lambda x: x.value)
    print(dice)
    gap = goal - total
    gap
    if gap<0:
        raise Exception
    for y in dice:
        y
        if y.value<=gap and y.cost<=money:
            return y
    return "Hold"

def StartHighSometimesBustStrategy(total, goal, money, dice):
    dice.sort(reverse=True, key=lambda x: x.value)
    print(dice)
    gap = goal - total
    gap
    if gap<0:
        raise Exception
    for y in dice:
        y
        if ((y.value*2)/3)<=gap and y.cost<=money:
            return y
    return "Hold"

def StartHighFrequentlyBustStrategy(total, goal, money, dice):
    dice.sort(reverse=True, key=lambda x: x.value)
    print(dice)
    gap = goal - total
    gap
    if gap<0:
        raise Exception
    for y in dice:
        y
        if (y.value/2)<=gap and y.cost<=money:
            return y
    return "Hold"

def StartLowStrategy(total, goal, money, dice):
    dice.sort(key=lambda x: x.value)
    print(dice)
    gap = goal - total
    print(gap)
    if gap<0:
        raise Exception
    for y in dice:
        print(y)
        if y.value<=gap and y.cost<=money:
            return y
    return "Hold"

def WildCardStrategy(total, goal, money, dice):
    print(dice)
    return random.choice(dice)
        

In [ ]:
def RunGame(player_1, player_2, dice_list, results_df, goal = 20, blind = 1, rounds=None):
    player_1 = copy(player_1)
    player_2 = copy(player_2)
    pot = 0
    round_number = 1
    print(dice_list)
    dice_list = copy(dice_list)
    initial_dice_list = copy(dice_list)
    if random.random()<0.5:
        player_order = [player_1, player_2]
    else:
        player_order = [player_2, player_1]
    player_1.active = True
    player_2.active = True
    while (rounds is None or round_number<=rounds) and (player_1.money>(10*blind) and player_2.money>(10*blind)):
        player_1.money = player_1.money - blind
        pot = pot + blind
        player_2.money = player_2.money - blind
        pot = pot + blind
        while len(dice_list)>0 and (player_1.active is True or player_2.active is True):
            for z in player_order:
                if z.active is True:
                    try:
                        die = z.strategy(z.total, goal, z.money, dice_list)
                        print("Die is {}".format(die))
                    except Exception as e:
                        z.active = False
                        z.total = 0
                        die = "Hold"
                        print("Player busted")
                        print(e)
                    if die != "Hold":
                        z.total = z.total + (random.randint(1, die.value))
                        if z.total>goal:
                            z.active = False
                            print("{} busted".format(z.name))
                        z.money = z.money - die.cost
                        pot = pot + die.cost
                        print(pot)
                        dice_list.remove(die)
                        print("Removing {} from dice list".format(die))
                        dice_list
                    else:
                        z.active = False
        if player_1.total> goal:
            player_1.total = 0
        if player_2.total > goal:
            player_2.total = 0
        if player_1.total == player_2.total:
            player_1.money = player_1.money + (pot/2)
            player_2.money = player_2.money + (pot/2)
            pot = 0
        if player_1.total > player_2.total:
            player_1.money = player_1.money + pot
            pot = 0
        if player_2.total > player_1.total:
            player_2.money = player_2.money + pot
            pot = 0
        results_dict = {"round": str(round_number), 
                        "player 1 strategy": player_1.strategy, 
                            "player 2 strategy": player_2.strategy, 
                            "player 1 result": player_1.total, 
                            "player 2 result": player_2.total, 
                            "player 1 money": player_1.money, 
                            "player 2 money": player_2.money}
        print(results_dict)
        results_df = results_df.append(results_dict, ignore_index=True)
        round_number = round_number + 1
        print("Round Number is {}".format(round_number))
        dice_list = copy(initial_dice_list)
        print(dice_list)
        player_1.total = 0
        player_2.total = 0
        player_1.active = True
        player_2.active = True
        player_order = player_order[::-1]
    return results_df


        

In [ ]:
start_high_never_bust = Player("Player 1", StartHighNeverBustStrategy, player_1_money, True)
start_low = Player("Player 2", StartLowStrategy, player_2_money, True)
start_high_sometimes_bust = Player("Player 3", StartHighSometimesBustStrategy, player_1_money, True)
start_high_frequently_bust = Player("Player 4", StartHighFrequentlyBustStrategy, player_2_money, True)
wild_card = Player("Player 5", WildCardStrategy, player_1_money, True)

In [ ]:
start_low.strategy(0, 20, 100, original_dice_list)

In [ ]:
results_1 = RunGame(start_high_never_bust, start_low, original_dice_list, data)
results_1

In [ ]:
results_2 = RunGame(start_high_never_bust, start_high_sometimes_bust, original_dice_list, data)
results_2

In [ ]:
results_3 = RunGame(start_high_never_bust, start_high_frequently_bust, original_dice_list, data)
results_3

In [ ]:
results_4 = RunGame(start_low, start_high_sometimes_bust, original_dice_list, data)
results_4

In [ ]:
results_5 = RunGame(start_low, start_high_frequently_bust, original_dice_list, data)
results_5

In [ ]:
results_6 = RunGame(start_low, wild_card, original_dice_list, data)
results_6

In [ ]:
results_7 = RunGame(start_high_never_bust, wild_card, original_dice_list, data)
results_7